# IBM's Python Project for Data Engineering

This project is part of the IBM Data Engineering Professional Certificate. It is intended for learners to apply basic Python knowledge to Extract, Transform and Load (ETL) data.

In this short course I've used Skills Network Labs's Cloud IDE as environment and I've used Python to write the code. Since Cloud IDE is not a persistent platform, I've saved a copy on my local machine and edited the code on Visual Studio Code, changing file paths accordingly to make it work. 
I've used Jupyter Notebook in order to show and explain the process.



## Tasks

The aim is to create a code that generates a dataframe containing the top 10 largest banks in the world ranked by market capitalization in billion USD. 
The data needs to be transformed and stored in GBP, EUR and INR as well. The dataframe is to be saved locally in a CSV format and as a database table.

Task 1:
Write a function to create log entries and save them in a txt file.

Task 2:
Perform web scraping with BeautifulSoup. Inspect the HTML code from the given URL, extract information from the table under the heading 'By market capitalization' and save it to a dataframe.
Write a function to perform the required data extraction.

Task 3:
Write a function to transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.

Task 4:
Write a function to load the transformed dataframe to an output CSV file.

Task 5:
Write a function to load the transformed dataframe to an SQL database server as a table. 

Task 6:
Run queries on the database table.

The data URL is https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks
The exchange rate CSV file path is https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

## Preliminaries
Before building the code, I need to install the required libraries.

The libraries needed for the code are:

`requests` - The library used for accessing the information from the URL.

`bs4` - The library containing the BeautifulSoup function used for webscraping.

`pandas` - The library used for processing the extracted data, storing it in the required formats, and communicating with the databases.

`numpy` - The library required for the mathematical rounding operations.

`sqlite3` - The library required to create a database server connection.

`datetime` - The library containing the function datetime used for extracting the timestamp for logging purposes.

While the modules `requests`, `sqlite3` and `datetime` are part of the Python standard library, I need to install the other required libraries from the terminal window.

Therefore, I run the following code on the terminal:

`pip install bs4`
`pip install pandas`
`pip install numpy`

From the terminal, I also need to download the required exchange rate file. I use the `cd` command to change directory, then run the following command:

`wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv`

Finally, I import the required libraries. 

In [ ]:
# Importing the required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

After importing the libraries I need, I create some variables:
- the URL I need to extract information from
- a list of attributes of the column names for the dataframe, one is the name of the bank and one is the Market capitalization in billion USD as listed in the table of interest from the webpage
- a list of attributes for the final dataframe, containing also the GBP, EUR and INR fields
- the database name
- the table name
- the target CSV file path 
- the log txt file

In [ ]:
data_url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
table_attribs_final = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"] 
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'
log_file = 'code_log.txt'

## Task 1: Logging function
Write the function to log the progress of the code, `log_progress()`. The function accepts the message to be logged and enters it to a text file `code_log.txt`.

The format to be used for logging must have the syntax:
`<time_stamp> : <message>`

Each log entry must happen in the next line in the text file.
Print the relevant logging message at the end of each associated function call (we will do this later).

| Task | Log message on completion |
|:----------------------------|:--------------------------------------------------------|
| Declaring known values | Preliminaries complete. Initiating ETL process |
| Call extract() function | Data extraction complete. Initiating Transformation process |
| Call transform() function	| Data transformation complete. Initiating Loading process |
| Call load_to_csv() | Data saved to CSV file |
| Initiate SQLite3 connection | SQL Connection initiated |
| Call load_to_db()	| Data loaded to Database as a table, Executing queries |
| Call run_query()	| Process Complete |
| Close SQLite3 connection | Server Connection closed |

In [ ]:
def log_progress(message):
    # This function logs the mentioned message at a given stage of the code execution to a log file
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) # convert timestamp format to string
    with open("./code_log.txt","a") as file: 
        file.write(timestamp + ' : ' + message + '\n') # add each log entry in a new line in the txt file

## Task 2: Extraction of data
Identify the position of the required table under the heading 'By market capitalization'. Write the function `extract()` to retrieve the information of the table to a Pandas data frame.

In [ ]:
def extract(data_url, table_attribs):
    # This function extracts the required information from the website and saves it to a dataframe
    # The function returns the dataframe for further processing
    page = requests.get(data_url).text # extract the webpage as text
    data = BeautifulSoup(page, 'html.parser') # parse the text into an HTML object
    df = pd.DataFrame(columns=table_attribs) # create a pandas DataFrame with column argument set as table_attribs
    data_dict = dict() # create an empty dictionary 
    Name_vals = [] # create an empty list that will contain the banks' names
    Market_cap_vals = [] # create an empty list that will contain the Market capitalization values
    tables = data.find_all('tbody') # find all the tables in the HTML code
    rows = tables[0].find_all('tr') # from table number 1 (at index 0), find all rows
    for row in rows:
        cells = row.find_all('td') # for each row, find all data cells
        for cell in cells:
            if len(cells) != 0: # check that entries are not empty
                Name = cells[1].find_all('a')[1].contents[0] # extract the text content of the index 1 cell, after the second link ('a' tag) 
                Market_cap = float(cells[2].contents[0][:-1]) # extract the content of index 2 cell, remove the last character, 
                # and typecast the value to float format
                if Name not in Name_vals: # we don't want any duplicates so we check if the bank name is already in the Name_vals list
                    Name_vals.append(Name)
                    Market_cap_vals.append(Market_cap)
                    data_dict = {"Name": Name, "MC_USD_Billion": Market_cap}
                    df1 = pd.DataFrame(data_dict, index=[0]) # create a dataframe from the data_dict dictionary 
                    df = pd.concat([df,df1], ignore_index=True) # combine dataframes
                else: 
                    continue
            else:
                continue
    return df

## Task 3: Transformation of data
1. Write the function `extract_from_csv()` to read the exchange rate CSV file and convert the contents to a dictionary so that the contents of the first column ('Currency') are the keys to the dictionary and the contents of the second column are the corresponding values ('Rate');
2. Write the function `transform()` and add 3 different columns to the dataframe: `MC_GBP_Billion`, `MC_EUR_Billion` and `MC_INR_Billion`, each containing the content of `MC_USD_Billion` scaled by the corresponding exchange rate factor. Round the resulting data to 2 decimal places.

In [ ]:
def extract_from_csv(file_to_process):
    csv_df = pd.read_csv(file_to_process, index_col=0)
    csv_dict = csv_df.to_dict()['Rate']
    return csv_dict

In [ ]:
def transform(df):
    # This function transforms the Market Capitalization information according to exchange rate in GBP, EUR and INR,
    # rounded to 2 decimal places, and adds the respective columns to the dataframe
    # The function returns the transformed dataframe
    df['MC_GBP_Billion'] = [np.round(x*csv_dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*csv_dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*csv_dict['INR'],2) for x in df['MC_USD_Billion']]
    return df


## Task 4: Loading to CSV
Write the function `load_to_csv()` to load the transformed data frame to a CSV file.

In [ ]:
def load_to_csv(df, csv_path):
    # This function saves the final dataframe as a CSV file in the provided path
    df.to_csv(csv_path)

## Task 5: Loading to Database
Write the function `load_to_db()` to load the transformed data frame to an SQL database.

Note: Before calling this function, we will initiate the connection to the SQLite3 database server with the name `Banks.db`. 

In [ ]:
def load_to_db(df, sql_connection, table_name):
    # This function saves the final dataframe as a database table
    df.to_sql(table_name, sql_connection, if_exists = 'replace', index = False)

## Task 6: Run queries on Database
Write the function `run_queries()` that accepts the query statement, and the SQLite3 Connection object, and generates the output of the query. 

In [ ]:
def run_query(query_statement, sql_connection):
    # This function runs the stated query on the database table and prints the output on the terminal 
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

# Let's do this!
Now let's write the function calls.

In [ ]:
# Extracting information
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(data_url, table_attribs)

# Transforming information
log_progress('Data extraction complete. Initiating Transformation process')

csv_dict = extract_from_csv('exchange_rate.csv')

df = transform(df)

# Loading information
log_progress('Data transformation complete. Initiating Loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db') # Connecting to the SQLite3 database server

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

# Querying the database table 
log_progress('Data loaded to Database as a table, Executing queries')

# Query no. 1: Print the contents of the entire table
query_statement1 = f"SELECT * FROM Largest_banks"
run_query(query_statement1, sql_connection)

# Query no. 2: Print the average market capitalization of all the banks in Billion GBP
query_statement2 = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement2, sql_connection)

# Query no. 3: Print only the names of the top 5 banks
query_statement3 = f"SELECT Name FROM Largest_banks LIMIT 5"
run_query(query_statement3, sql_connection)

log_progress('Process Complete.')

sql_connection.close() # Closing SQLite3 connection

log_progress('Server Connection closed')

Done!